# Introdução

O problema consiste na validação de um log de dados ambientais de uma estação situada em Petrolina. Como os dados correspondem a mais de 10 anos, uma simples planilha não seria eficiente (O excel suporta apenas ~1Mi de linhas). 

Foi selecionada a linguagem python, onde podemos usar a biblioteca pandas para análise de dados.

In [ ]:
# importando a biblioteca com prefixo pd
import pandas as pd

Assumindo que estamos no diretório do log, podemos ler o csv com a função `pd.read_csv()`. Atenção para não rodar a leitura múltiplas vezes, visto que o arquivo é grande e será carregado em memória.

In [ ]:
csv = pd.read_csv('Petrolina.csv')

In [ ]:
# O objeto criado é um dataframe, semelhante a uma planilha.
type(csv)

# Visualizando a estrutura dos dados

a biblioteca `pandas` é bastante versátil, permitindo rápidas análises. 
Podemos ver _head_ e _tail_ do _dataframe_ apenas lendo o objeto. 
Isso já nos permite entender com o que estamos lidando.

In [ ]:
print(f'O log possui {csv.shape[0]:,} linhas e {csv.shape[1]:,} colunas')

csv

já podemos ver que colunas como **GHI, DNI** e **DHI** possuem valores negativos, o que é fisicamente impossível. Provavelmente o datalogger capturou valores noturnos.

Outro problema é o formato da coluna Date que está como _string_. Convertê-la para objeto _datetime_ facilita muito durante análises de séries temporais.

In [ ]:
type(csv['Date'][0])

In [ ]:
# discriminar o formato da coluna previamente aumenta massivamente
# o tempo de processamento, não fazer é uma má prática

csv['Date'] = pd.to_datetime(csv['Date'], format="%d-%b-%Y %H:%M:%S")

In [ ]:
type(csv['Date'][0])

Outra melhoria em séries temporais é [usar um índice de datas](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#indexing).

In [ ]:
csv.set_index('Date', inplace = True)

In [ ]:
# agora não temos mais o índice numérico
csv

## Extremos
ver os extremos dos dados pode ajudar em reconhecer _outliers_

In [ ]:
csv.nlargest(10, 'GHI')

In [ ]:
csv.nsmallest(10, 'GHI')

# Caracterização

Por padrão, tem-se o seguinte modelo para avaliar os dados ruins de nosso log:

|#|flag|significado|exemplo|
|:---|:---|:---|---:|
|1|bom| foi aprovado em todos os testes |ideal|
|2|suspeito| teste não permite concluir se é correto ou é um outlier||
|3|anômalo| não será avaliado em nenhum dos testes posteriores||
|4|previamente anômalo| teste anterior caracterizou-o como anômalo||
|5|dado não testado| não será avaliado em qualquer teste do procedimento.| dados noturnos|
|6|dado não disponível| não foi registrado pelo sistema de aquisição de dados| NaN|

## Testes de Lacunas

- [ ] Descontinuidade
- [ ] Missing data
- [ ] Duplicidade de datas
- [ ] Lacunas

In [ ]:
# resetar o índice para poder diferenciar os timestamps
# e depois agrupar-los
csv.reset_index().diff()

In [ ]:
csv.reset_index().diff().groupby('Date').count()

Como em toda série temporal só temos intervalos de 1 minuto, conclui-se que não há descontinuidadem duplicidade ou lacunas.
- [x] Descontinuidade
- [ ] Missing data
- [x] Duplicidade de datas
- [x] Lacunas

In [ ]:
qtdNan = csv['GHI'].isna().sum()

print(f'são {qtdNan:,} dados GHI ausentes que representam',
      f'cerca de {qtdNan/len(csv)*100:.3}% do total')

csv[csv['GHI'].isnull()]

In [ ]:
csv['NaN'] = csv['GHI'].isna()

In [ ]:
csv

- [x] Descontinuidade
- [x] Missing data
- [x] Duplicidade de datas
- [x] Lacunas

In [ ]:
negativos = csv['GHI'] < 0
qtdNegativos = csv[negativos]['GHI'].count()

print(f'são {qtdNegativos:,} dados GHI negativos que representam',
      f'cerca de {qtdNegativos/len(csv)*100:.4}% do total')

csv[negativos]

In [ ]:
print(f"Ao todo, já podemos ver a presença de {qtdNan + qtdNegativos:,}",
      " dados sem utilidade na avaliação do recurso solar,",  
      f"representando {(qtdNan + qtdNegativos) * 100 / csv.shape[0]:.3}% do total.")

## Testes Locais

In [1]:
import solar

Para a análise de _Tracker off_ é necessário $\cos\theta$. Entretanto, uma primeira análise pode ser feita desconsiderando as seguintes situações:

$\frac{I_d}{I_g} > 1$
ou 
$\frac{I_{bn}}{I_g} > 1$

In [ ]:
csv['Tracker off'] = (csv['DHI'])/csv['GHI']

# Limpando e visualizando

In [ ]:
# criar uma coluna com o ano de cada timestamp
# para poder ver o boxplot de cada ano

csv['ano'] = csv.index.year
csv.boxplot(by='ano', column='GHI')

Apesar de ser possível visualizar, os dados estão mal representados por causa de _bad data_. Deve-se avaliar depois de limpar.

In [ ]:
# limpar os valores negativos

csv.drop(csv[csv['GHI'] < 0].index, inplace=True)

# limpar NaN's

csv.dropna(subset=['GHI'], inplace=True)

In [ ]:
# novamente
csv.boxplot(by='ano', column='GHI')

In [ ]:
print(f'no momento, tem-se {len(csv)/5_785_920*100:.4}% dos dados iniciais')

In [ ]:
csv
csv.boxplot(by='ano', column='Tracker off')

In [ ]:
csv.drop(csv[csv['Tracker off'] > 1].index, inplace=True)

In [ ]:
csv
csv.boxplot(by='ano', column='Tracker off')

In [ ]:
csv['Suspeito'] = (csv['Tracker off'] > 0.85)
csv['Suspeito'].sum()